# Introducción a PySpark

En este primer notebook utilizaremos Spark de forma local a través de PySpark. El objetivo es empezar a conocer la filosofía de su funcionamiento através de los comandos básicos.

### Instalación
Tenemos varias opciones para tilizar Spark de forma local:

1. En nuestra PC, crear un ambiente nuevo de python e instalar Spark, por ejemplo utilizando conda:
    * Crear el ambiente:
    __conda create -n env_pyspark__
    * Activar el ambiente:
    __conda activate env_pyspark__
    * Instalar pyspark:
    __conda install -c conda-forge pyspark ipykernel__
    * (opcional) Instalar librerias extra

2. Utilizarlo en colab e instalarlo de la siguiente manera:
__!pip install pyspark__


In [ ]:
!pip install pyspark

### Iniciar una sesión en Spark

El primer paso al trabajar con Spark es iniciar una sesión. Lo podemos hacer de la siguiente forma:

In [ ]:
#--importamos el objeto para cargar la sesion
from pyspark.sql import SparkSession

#--inicializamos la sesion
spark = (SparkSession.builder
  #-especificamos que es local y que utilizamos todos los cores disponibles *
  .master('local[*]')
  #-podemos asignar un nombre a la sesión
  .appName('hello_world_spark')
  .getOrCreate())

#--probamos que spark se haya inicializado
spark

## Spark DataFrames

PySpark dataframes son, a simple vista, similares a los de Pandas. Sin embargo a bajo nivel su implementación es totalmente diferente, su construcción esta basada en RDDs.

Todas las transformaciones que se aplican a un dataframe son evaluadas de manera "lazy" hasta que alguna __acción__ se manda a llamar explícitamente. Esto permite que se concatenen varias transformaciones y se optimicen en conjunto, para llegar al resultado final.

__Definicion__ dataframe en spark: tablas distribuidas en memoria cuya construcción contiene un esquema y nombres de columnas.

Cada columna guarda un tipo especifico de dato, como: entero, character, array, etc.

### Crear un data frame

In [ ]:
#--importamos librerías de python para definir fechas
from datetime import datetime, date
#--objeto de pyspark para definir filas de un dataframe
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2.0, c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3.0, c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])

#--imprimimos
df

El __schema__ es la parte del dataframe que define el nombre de las columnas y su tipo. Sino lo especificamos, spark automaticamente tratara de inferirlo.

Es recomendable especificar el __schema__ siempre que sea posible porque:

* Es computacionalmente costoso inferirlo en dataframes "grandes".
* Ayuda a detectar error: si un data es de un tipo diferente al esperado.

Existen dos formas de definir un "schema", uno alineado a la formalidad de la programación y otro alienado más a la semantica humana. Un ejemplo de segundo seria:

&nbsp;&nbsp;&nbsp; schema="name_col1 TIPO_DATO, name_col1 TIPO_DATO"

A continuación vemos la creación del mismo data frame con definición de "schema".

In [ ]:
df = spark.createDataFrame([
    (1, 2.0, 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3.0, 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
],
    schema='a LONG, b DOUBLE, c STRING, d DATE, e TIMESTAMP')
df

Otra forma de crear un data frame en spark es importarlo desde pandas o desde archivos RDD:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;__spark.createDataFrame(pandas_df)__

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;__spark.createDataFrame(rdd_list_tuples)__

y finalmente podemos guardar un __spark dataframe__ de regreso a pandas con:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;__df.toPandas()__

### Visualización de un dataframe

Para visualizar un dataframe y su esquema podemos usar:

In [ ]:
df.show(2)

Si queremos ver el dataframe de forma vertical:

In [ ]:
#--si alguna columna tiene información mucha información, podemos imprimir de forma vertical
df.show(1, vertical=True)

In [ ]:
#--para visualizar el esquema
df.printSchema()

Al igual que en pandas, tambien existe en spark la funcion __describe()__ y __summary()__ la cual nos da un resumen de las estadísticas del dataset:

In [ ]:
#--"describe" solo recibe como argumento las columnas a las cuales calcular
# la descripción
#df.describe(['a', 'b']).show()

#--"summary" recibe como argumento las estadísticas a calcular
#df.summary(['mean', '99%', 'max']).show()

#--podemos utilizar "select" para seleccionar las columnas antes del summary
df.select(['a','b']).summary(['mean', '95%', '99%', 'max']).show()

"show()" es un método solamente para __imprimir__. Si deseamos hacer una rebanada (slice) de las primeras n filas o las últimas n filas podemos utilizar:

* __take()__
* __tail()__
* __first()__
* __limit()__

In [ ]:
#--para las primeras n filas
df.take(3)

In [ ]:
#--para las últimas n filas
df.tail(2)

In [ ]:
#--también existe el método "first" para traer la primera fila que no es nula
df.first()

In [ ]:
#--A diferencia de las demás, nos regresa un dataFrame
df.limit(4).show()

Existe una acción más llamada __collect()__, la cual concentra todo los resultados de la transformación y las trae a la memoria local. Debe utilizarse con cuidado porque si el dataset es demasiado grande habrá un desbordamiento de memoria.

Esta instrucción es útil cuando calculamos sumarizados de los datos y los deseamos traer a la memoria local para analizarlos o hacer gráficas.

In [ ]:
df.collect()

### Selección de subsets en un dataframe

En esta sección ejemplificamos como obtener un subset de columnas y/o filas del dataset.

Estas transformaciones son evaluadas de forma "lazy" hasta que una acción, e.g. "show", es mandada a llamar

In [ ]:
#--accesar al nombre de las columnas
df.columns

En PySpark las columnas son objetos con métodos públicos. Se representan con el tipo: __Column__. Aunque cada columna es un objeto, no puede exister por si misma. Cada columna es parte de un fila, y todas las filas constituyen un dataframe.

La selección de columnas la podemos realizar:

In [ ]:
#--acceder a una sola columna
df.a

#--básicamente esta notación la utilizamos cuanda hacemos una
# selección anidada, por ejemplo para filtrar filas usando
# condicionales

La otra forma es usando el método __select__:

In [ ]:
#--la siguiente línea imprime el resultado debido a que tenemos activo
#     spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
# Esta característica solo esta disponible en notebooks
#df.select('a')

# Si quisieramos imprimir en otro ambiente que no sea notebooks, utilizariamos:
#df.select('a').show()

# Si son varias columnas, utilizamos una lista
# df.select(['a','c'])

# También, podemos utilizar "my_dataFrame.nombre_columna" dentro de "select":
df.select(df.a).show()

Selección de un subset de filas se puede hacer utilizando __filter__ o __where__:

In [ ]:
#--usando filter
#df.filter(df.a == 1)
df.filter(df.a == 1).show()

In [ ]:
#--usando where
df.where(df.a == 1).show()

### Agregar nuevas columnas al dataframe

Para agregar columnas se utiliza la funcion __withColumn__, el primer argumento es el nombre de la nueva columna y el segundo, la columna a agregar.

Si utilizamos el nombre de una columna existente, la columna se sustituye.

In [ ]:
from pyspark.sql.functions import upper

#--agregamos una columna que contenga la misma informacion que "c" pero en
#-letras mayusculas
df.withColumn('upper_c', upper(df.c)).show()

El modulo "__pyspark.sql.functions__" contiene muchas de las funciones que utilizamos para procesar los datos. Es una convención utilizar el siguiente import:

In [ ]:
import pyspark.sql.functions as F

Otra forma de agregar columnas es con "select":

In [ ]:
df.select('*', F.upper(df.c).alias('upper_c')).show()

¿Cuál es la diferencia entre las dos formas?

Podemos ver las diferencias en el plan que Spark ejecutara utilizado el comando __explain()__:

In [ ]:
df.select('*', upper(df.c).alias('upper_c')).explain()

In [ ]:
df.withColumn('upper_c', upper(df.c)).explain()

In [ ]:
df.show()

In [ ]:
# cast double a int
double_cols = ['a', 'b', 'c']
my_df = df
for my_col in double_cols:
  my_df = (my_df
           .withColumn(my_col, F.upper(F.col(my_col))))

my_df.explain()


Es posible enlazar transformaciones y acciones. Por ejemplo, considere las siguientes acciones:

* Agregar una columna nueva: concatenación de otras columnas
* Seleccionar la columna recién creada
* Mostrar las primeras dos filas

In [ ]:
#-- expr: toma como argumento un string y lo convierte en nombre de columna
(df
 #-crear una nueva columna 'dca' que es la concatenacion de las columnas
 .withColumn('dca', (F.concat(F.expr('d'), df.c, F.expr('a *2'))))\
 .select('dca')
 .show(2))


### Leer y escribir archivos

Spark ofrece varias opciones para leer archivos desde diferentes fuentes y formatos.

Los archivos CSV se puede guardar y leer de la siguiente forma:

In [ ]:
#--guardar un archivo a csv
df.write.csv('foo.csv', header=True, mode="overwrite")

La celda anterior guarda el archivo en un CSV que es distribuido. Si deseamos obtener un csv tradicional, tenemos que convertirlo a pandas dataframe:

In [ ]:
import pandas as pd

df.toPandas().to_csv('foo_pandas.csv', index=False)

Para leer de CSV:

In [ ]:
#--importante definir el "schema"
df = spark.read.csv('foo.csv', header =True,
                    schema='a LONG, b DOUBLE, c STRING, d DATE, e TIMESTAMP')

df.show()

Las primeras etapas de un proyecto de ciencia de datos consisten en explorar, limpiar y transformar los datos. Es recomendable, especialmente en grandes cantidades de datos, guardar una copia del dataset transformado para mantener los cambios y no calcularlos cada vez que iteramos.

Aunque los CSV es un formato común __no es eficiente para leer y esribir en disco__. Por tanto, se recomienda utilizar formatos optimizados para estos fines como lo es el formato "parquet".

__Notas__:
* Parquet no admite nombre de columnas con espacios.
* Cuando salvamos a parquet, se guarda el "schema". No es necesario volver a definirlo al leer de parquet.

In [ ]:
#--escribir nuestro dataframe a un archivo parquet
df.write.parquet('bar.parquet', mode="overwrite")

#--volver a leerlo
spark.read.parquet('bar.parquet').show()

### SQL

Spark SQL y dataframe funcionana sobre la misma infraestructura por lo que pueden utilizarse de forma intercambiable.

Para utilizar SQL-queries creamos una "vista temporal" de nuestros datos y después podemos hacer "queries" como si fuera SQL.

In [ ]:
#--vista temporal
df.createOrReplaceTempView("table_A")

#--query
spark.sql("SELECT count(*) from table_A").show()

Otra forma es utilizar "expr" y el query que deseamos hacer:

In [ ]:
df.select(F.expr('count(*)')).show()

#--debido a que "select" y "expr" son muy utilizadas, existe un forma más corta:
df.selectExpr('count(*)').show()

## Ejemplo 1: contar chocolates m&m

Vamos a leer un archivo CSV que contiene la cuenta de chocolates m&m por color y por estado.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

mnm_file = "/content/drive/MyDrive/data_sets/mnm_dataset.csv"
mnm_schema = 'State string, Color string, Count int'

Otra forma distinta de leer CSV.

In [ ]:
#--leer el archivo csv
#-especificamos que es "csv", le decimos que tiene nombre de columnas
mnm_df = (spark.read.format("csv")
    .option("header", "true")
    # .option("inferSchema", "true")
    .option('schema', mnm_schema)
    .load(mnm_file))

In [ ]:
mnm_df.show(5)

Supongamos que deseamos contar los m&m por estado y por color:

In [ ]:
# 1. Seleccionamos las columnas de interes
# 2. agrupamos por estado y color
# 3. agregamos la cuenta
# 4. ordenamos de forma descendete

#--en este ejemplo, utilizo () para agrupar todas las acciones
count_mnm_df = (mnm_df
    .select("State", "Color", "Count")
    .groupBy("State", "Color")
    .agg(F.sum("Count").alias("Total"))
    .orderBy("Total", ascending=False))

#--imprimimos los resultados
count_mnm_df.show(n=10, truncate=False)
print(f"Total Rows = {count_mnm_df.count()}")

Todo procesamiento se puede hacer también utilizando sintaxis de SQL. Se debe crear una "vista" de la tabla y ya después se puede operar sobre esa vista.

In [ ]:
#--repetir el ejercicio anterior utilizando sintaxis SQL
mnm_df.createOrReplaceTempView("table_mnm")
spark.sql("SELECT State, Color, count(*) as COUNT from table_mnm group by State, Color order by COUNT DESC").show()

Si solo deseamos saber el resultado del estado de California:

In [ ]:
#--asumamos que deseamos ver solo datos de un solo estado
ca_count_mnm_df = mnm_df\
    .select("State", "Color", "Count")\
    .where(mnm_df.State == "CA")\
    .groupBy("State", "Color")\
    .agg(F.count("Count").alias("Total"))\
    .orderBy("Total", ascending=False)

#--imprimimos
ca_count_mnm_df.show(n=5, truncate=False)

## Ejemplo 2: San Francisco fire calls

De nuevo vamos a leer un CSV, sin embargo esta vez vamos a definir el "schema" siguiendo una forma más estricta:

In [ ]:
from pyspark.sql.types import *

#--otra forma de definir un scheme
fire_schema = StructType([StructField(name='CallNumber', dataType=IntegerType(), nullable=True),
    StructField('UnitID', StringType(), True),
    StructField('IncidentNumber', IntegerType(), True),
    StructField('CallType', StringType(), True),
    StructField('CallDate', StringType(), True),
    StructField('WatchDate', StringType(), True),
    StructField('CallFinalDisposition', StringType(), True),
    StructField('AvailableDtTm', StringType(), True),
    StructField('Address', StringType(), True),
    StructField('City', StringType(), True),
    StructField('Zipcode', IntegerType(), True),
    StructField('Battalion', StringType(), True),
    StructField('StationArea', StringType(), True),
    StructField('Box', StringType(), True),
    StructField('OriginalPriority', StringType(), True),
    StructField('Priority', StringType(), True),
    StructField('FinalPriority', IntegerType(), True),
    StructField('ALSUnit', BooleanType(), True),
    StructField('CallTypeGroup', StringType(), True),
    StructField('NumAlarms', IntegerType(), True),
    StructField('UnitType', StringType(), True),
    StructField('UnitSequenceInCallDispatch', IntegerType(), True),
    StructField('FirePreventionDistrict', StringType(), True),
    StructField('SupervisorDistrict', StringType(), True),
    StructField('Neighborhood', StringType(), True),
    StructField('Location', StringType(), True),
    StructField('RowID', StringType(), True),
    StructField('Delay', FloatType(), True)])

Leemos los datos

In [ ]:
sf_fire_file = "/content/drive/MyDrive/data_sets/sf-fire-calls.csv"
fire_df = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)
fire_df.show(5, truncate=False)

Asumimos que hicimos algún preprocesamiento a los datos, y vamos a proceser a guardar los cambios. Finalmente, leemos de nuevo el archivo.

In [ ]:
#--definimos donde queremos guarar el archivo
parquet_path = "/content/drive/MyDrive/data_sets/sf_fire_calls"
#--lo guardamos
fire_df.write.save(parquet_path, format="parquet", mode="overwrite")

In [ ]:
#--volvemos a leer los datos, ahora desde el formato parquet
#--No necesitamos especificar el schema debido a que el formato parquet lo guarda
fire_df = spark.read.parquet(parquet_path)

Selección de columnas y filas especificas la podemos hacer da l siguiente manera:

In [ ]:
few_fire_df = (fire_df
    .select("IncidentNumber", "AvailableDtTm", "CallType")
    .where(F.col("CallType") != "Medical Incident"))

few_fire_df.show(5)

¿Cómo podríamos saber cuantos tipos de llamadas (CallTypes) fueron hechas?

In [ ]:
(fire_df
    .select("CallType") #-seleccionamos la columna 'CallType'
    .where(F.col("CallType").isNotNull()) #-obtenemos solo las filas que no son nulas
    .agg(F.countDistinct("CallType").alias("TiposDellamadas")) #-contamos cuantos registros hay
    .show())

Podemos listar los tipos de llama usando la función "dropDuplicates". A continuación solo mostramos las primeras 10:

In [ ]:
(fire_df
    .select("CallType")
    .where(F.col("CallType").isNotNull())
    .dropDuplicates()
    .show(10, truncate=False))

El nombre de las columnas se decidió al definir el "schema". Si deseamos cambiar un nombre de columna lo podemos lograr con:

In [ ]:
#--cambiamos el nombre de la columna "Delay" por el de "ResponseDelayedinMins"
new_fire_df = fire_df.withColumnRenamed("Delay", "ResponseDelayedinMins")

#--seleccionamos las filas que tengan un retraso mayor a 5 mins e imprimimos las primeras 5 con
#-mayor retraso.
(new_fire_df
    .select("ResponseDelayedinMins")
    .where(F.col("ResponseDelayedinMins") > 5)
    .orderBy(F.col("ResponseDelayedinMins"), ascending=False)
    .show(5, False))

Algunas veces es necesario cambiar el tipo de dato de alguna columna. Como por ejemplo, las columnas "CallDate", "WatchDate" y "AvailableDtTm" son de tipo string pero contienen fechas.

In [ ]:
#--imprimimos que tipo de datos son las columnas
(new_fire_df
 .select('CallDate', 'WatchDate', 'AvailableDtTm')
 .printSchema())
#--imprimimos las primeras 5 filas
(new_fire_df
 .select('CallDate', 'WatchDate', 'AvailableDtTm')
 .show(5,False))


In [ ]:
fire_ts_df = (new_fire_df
    .withColumn(colName="IncidentDate", col=F.to_timestamp(F.col("CallDate"), "MM/dd/yyyy"))
    .drop("CallDate")
    .withColumn("OnWatchDate", F.to_timestamp(F.col("WatchDate"), "MM/dd/yyyy"))
    .drop("WatchDate")
    .withColumn("AvailableDtTS", F.to_timestamp(F.col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a"))
    .drop("AvailableDtTm"))

#--mostramos las columnas que cambiamos
(fire_ts_df
.select("IncidentDate", "OnWatchDate", "AvailableDtTS")
.show(5, False))

(fire_ts_df
.select("IncidentDate", "OnWatchDate", "AvailableDtTS")
.printSchema())

Con las columnas de tipo fecha podemos utilizar funciones como __month()__, __year()__, y __day()__. Por ejemplo, podemos ver todos los años donde se ha reportado algún incidente:

In [ ]:
(fire_ts_df
    .select(F.year('IncidentDate'))
    .distinct()
    #.orderBy(F.year('IncidentDate').desc())
    .orderBy(F.year('IncidentDate').asc())
    .show())

¿Cuáles es el tipo de incidente más común?

In [ ]:
(fire_ts_df
    .select("CallType") #-seleccionamos solo la columna de interes
    .where(F.col("CallType").isNotNull()) #-seleccionamos las filas no nulas
    .groupBy("CallType") #-las agrupamos por su tipo
    .count() #-contamos cuantas hay de cada tipo
    .orderBy("count", ascending=False) #-ordenamos
    .show(n=10, truncate=False)) #-imprimimos

Otras funciones comunes son __min()__, __max()__, __sum()__ y __avg()__. Para un mayor detalle ver la API "pyspark.sql.functions". Un ejemplo:

In [ ]:
(fire_ts_df
    .select(F.sum("NumAlarms"), F.avg("ResponseDelayedinMins"),
            F.min("ResponseDelayedinMins"), F.max("ResponseDelayedinMins"))
    .show())

Algunas otras funciones más especializadas:

* abs()
* stat()
* describe()
* correlation()
* covariance()
* sampleBy()
* approxQuantile()
* frequentItems()

Para mas información:

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

Una vez que terminamos de utilizar Spark es importante cerrar la sesion para liberar los recursos que se estan ocupando.

In [ ]:
spark.stop()

In [ ]:
spark